<a href="https://colab.research.google.com/github/Khorrlybobo/Comparing-BART-and-BioBERT-for-Medical-Article-Summarization/blob/main/Comparing_BART_and_BioBERT_for_Medical_Article_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install pdfminer.six docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3957 sha256=19b5f29387da001c1c958fdb0af2d4661f7ea05b7051acafa3b51e38f5accfc8
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [2]:
from transformers import pipeline 
from pdfminer.high_level import extract_text 
import docx2txt

In [3]:
sumarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
def text_extraction(file):
  """
  To extract texts from both pdf and word
  """
  if file.endswith(".pdf"):
    return extract_text(file)
  else:
    resume_text = docx2txt.process(file)
  if resume_text:
    return resume_text.replace('\t', ' ')
  return None

In [5]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/test.pdf"
text_extracted = text_extraction(path)
print(len(text_extracted))

Mounted at /content/drive
2872


In [6]:
sumarizer(text_extracted, max_length =715, min_length = 30, do_sample = False)

[{'summary_text': ' Human embryo culture under 2-8% O2 is recommended by ESHRE revised guidelines for good practices in IVF labs . Recent meta-analyses concluded with low evidence that hypoxia has a superiority on IVF/ICSI outcomes . We aimed to demonstrate for the ﬁrst time in human embryos that OS has a negative impact only at cleavage stage .'}]

In [7]:
summarizer = pipeline("summarization", model="dmis-lab/biobert-large-cased-v1.1-mnli")


The model 'BertForSequenceClassification' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [8]:
sumarizer(text_extracted, max_length =715, min_length = 30, do_sample = False)

[{'summary_text': ' Human embryo culture under 2-8% O2 is recommended by ESHRE revised guidelines for good practices in IVF labs . Recent meta-analyses concluded with low evidence that hypoxia has a superiority on IVF/ICSI outcomes . We aimed to demonstrate for the ﬁrst time in human embryos that OS has a negative impact only at cleavage stage .'}]

In [ ]:
!pip install torch transformers
import torch 
from transformers import BertTokenizer, BertForMaskedLM


# Load BioBert model and tokenizer
model_name = 'dmis-lab/biobert-large-cased-v1.1'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Preprocess and tokenize the article text
def preprocess_text(text):
    # Remove unwanted characters or symbols
    cleaned_text = text.replace('\n', ' ').replace('\r', '')
    
    # Convert to lowercase
    cleaned_text = cleaned_text.lower()
    
    return cleaned_text

preprocessed_text = preprocess_text(text_extracted)  # Your own preprocessing function
tokens = tokenizer.encode(preprocessed_text, truncation=True, padding=True, return_tensors='pt', max_length=50, add_special_tokens = True)
token_type_ids = torch.zeros_like(tokens)  # Set token type IDs to 0


# Generate the summary
summary_ids = model.generate(tokens, max_length=1024, min_length=30, do_sample=False)
summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
